# Reading FAME databases from Python

This example demonstrates reading FAME data objects into Pandas structures.

FAME databases map data object names to data structures.  FAME data types include `BOOLEAN`, `DATE` (at various frequencies), `NUMERIC`, `PRECISION`, and `STRING`. The `pyhli` package maps FAME data types to compatible basic Python data types. The `qomautils` package further maps to more complex Pandas data structures such as `DatetimeIndex`, `Series`, and `DataFrame`.

In [1]:
import os
from __future__ import print_function
import pandas as pd
import qoma_smuggler as qm

The Qoma utility function `open_hli()` opens the FAME environment and prints diagnostic information.

In [2]:
if qm.open_hli()!=0:
    raise

Linux 4.14.33+ (#1 SMP Wed Jun 20 01:15:52 PDT 2018) x86_64
Python 2.7.5 (default, Apr 11 2018, 07:36:10) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-28)]
NumPy 1.14.5 Pandas 0.23.3 FAME HLI 11.63000 pyhli 0.0.6


The `qomautils` function `read_fame()` reads FAME data objects into a nested Python dictionary.  
At the top level, each FAME object name is mapped to a dictionary with entries `data` and `fame`.  
*  As appropriate, the entry `data` maps to a single data value or to multiple data values in an array.  
    *  For FAME `SCALAR` objects, `data` maps to a value. 
    *  For FAME `SERIES` objects, `data` maps to an array of values.
*  The entry `fame` maps to FAME object meta data such as object class (`SCALAR` or `SERIES`), object data type, and index values for `data`.

The FAME distribution includes a number of sample databases.  We will use the `driecon` sample database for this notebook.

In [3]:
dbname = os.path.join(os.environ['FAME'],'util','driecon')

famedata = qm.read_fame(dbname)
print("read_fame() returned {0} FAME objects from {1}.\n".format(len(famedata),dbname))

read_fame() returned 55 FAME objects from /opt/pkg/fame116linux/util/driecon.



The `print_fame_cata()` function in `qomautils` package is similar to the FAME 4GL command `CATALOG`.  We use `print_fame_cata()` to display a description of the contents of the Python dictionary `famedata` obtained wtih the `read_fame()` call.

In [4]:
qm.print_catalog(famedata)

SERIES RXC132%USNS : PRECISION BY DATE(MONTHLY) Nov1967 to Dec1998 -- EXCHANGE RATE IN NEW YORK - FRENCH FRANC PER US DOLLAR (G5)
EXCHANGE RATE IN NEW YORK - FRENCH FRANC PER US DOLLAR (G5)
AVG OF CERTIFIED NOON BUYING RATES FOR CABLE TRANSFERS, UNADJUSTED
BOARD OF GOVERNORS OF THE FEDERAL RESERVE
STATISTICAL RELEASE G.5 AND 'FEDERAL RESERVE BULLETIN'
DATA PRIOR TO NOVEMBER 1981 CALCULATED BY GII

SERIES HUCMPNC : PRECISION BY DATE(MONTHLY) Jan1979 to Jul2003 -- HOUSING COMPLETIONS, PRIVATE - NORTH CENTRAL (NRC)
HOUSING COMPLETIONS, PRIVATE - NORTH CENTRAL (NRC)
THOUSANDS OF UNITS, SEASONALLY ADJUSTED AT ANNUAL RATES
U.S. DEPARTMENT OF COMMERCE AND OF HOUSING AND URBAN DEVELOPMENT
STATISTICAL REALEASE: NEW RESIDENTIAL CONSTRUCTION
17,000 PLACES THEREAFTER

SERIES GNP : PRECISION BY DATE(ANNUAL) 1929 to 2002 -- GROSS NATIONAL PRODUCT
GROSS NATIONAL PRODUCT
BILLIONS OF CURRENT DOLLARS
U.S. DEPARTMENT OF COMMERCE, BUREAU OF ECONOMIC ANALYSIS
'SURVEY OF CURRENT BUSINESS' AND OTHER MATERIAL

We can be more selective in what we retrieve from a FAME database.  Here we specify a wildcard pattern "IP?". The Python dictionary `famedata` will now only include the industrial production time series in `driecon` with object name prefix "IP".

In [5]:
famedata = qm.read_fame(dbname,wilnam="IP?")
print("read_fame() returned {0} FAME objects from {1}.\n".format(len(famedata),dbname))
qm.print_catalog(famedata)

read_fame() returned 9 FAME objects from /opt/pkg/fame116linux/util/driecon.

SERIES IPSB51214 : PRECISION BY DATE(MONTHLY) Jan1954 to Jul2003 -- INDUSTRIAL PRODUCTION INDEX -  PAPER PRODUCTS
INDUSTRIAL PRODUCTION INDEX -  PAPER PRODUCTS
UNITS 1997=100, SEASONALLY ADJUSTED
FRB, INDUSTRIAL PRODUCTION, G.17;

SERIES IPSG311 : PRECISION BY DATE(MONTHLY) Jan1972 to Jul2003 -- INDUSTRIAL PRODUCTION INDEX -  FOOD  NAICS=311
INDUSTRIAL PRODUCTION INDEX -  FOOD  NAICS=311
UNITS 1997=100, SEASONALLY ADJUSTED
FRB, INDUSTRIAL PRODUCTION, G.17;

SERIES IPSG339 : PRECISION BY DATE(MONTHLY) Jan1972 to Jul2003 -- INDUSTRIAL PRODUCTION INDEX -  MISCELLANEOUS  NAICS=339
INDUSTRIAL PRODUCTION INDEX -  MISCELLANEOUS  NAICS=339
UNITS 1997=100, SEASONALLY ADJUSTED
FRB, INDUSTRIAL PRODUCTION, G.17;

SERIES IPSG336 : PRECISION BY DATE(MONTHLY) Jan1972 to Jul2003 -- INDUSTRIAL PRODUCTION INDEX -  TRANSPORTATION EQUIPMENT  NAICS=336
INDUSTRIAL PRODUCTION INDEX -  TRANSPORTATION EQUIPMENT  NAICS=336
UNITS 1997=

Some of the time series are quite long.  If we are only interested in a particular time window, we may specify a FAME date range.  Given Pandas data range `prng`, we convert to FAME date range `frng` using the `qomautils` function `to_fame_range()`.  We pass the FAME range to `read_fame()` as a parameter named `fame_range`.

Note in Python and Pandas, the data range _excludes_ the end point.  In FAME, the end point is included in the date range.  `to_fame_range()` and `to_pandas_range()` handle this distinction automatically.

In [6]:
# pandas range
prng = pd.date_range(start='1998', end='2003', freq='A')

# fame range
frng = qm.to_fame_range(prng)

famedata = qm.read_fame(dbname,fame_range=frng)
print("read_fame() returned {0} FAME objects.\n".format(len(famedata)))
qm.print_catalog(famedata)

read_fame() returned 16 FAME objects.

SERIES $N : PRECISION BY DATE(ANNUAL) 1998 to 2002 -- POPULATION INCLUDING ARMED FORCES OVERSEAS (P25E)
POPULATION INCLUDING ARMED FORCES OVERSEAS (P25E)
MILLIONS OF PERSONS, ESTIMATES ARE FOR JULY 1
U.S. DEPARTMENT OF COMMERCE, BUREAU OF THE CENSUS
CURRENT POPULATION REPORTS, SERIES P-25
INCLUDING ALASKA AND HAWAII BEGINNING IN 1940

SERIES GDP : PRECISION BY DATE(ANNUAL) 1998 to 2002 -- GROSS DOMESTIC PRODUCT
GROSS DOMESTIC PRODUCT
BILLIONS OF CURRENT DOLLARS
U.S. DEPARTMENT OF COMMERCE, BUREAU OF ECONOMIC ANALYSIS
'SURVEY OF CURRENT BUSINESS' AND OTHER MATERIALS
U.S. NATIONAL INCOME & PRODUCT ACCOUNTS - TABLE 1.1,1.3,1.5,1.9

SERIES GNP : PRECISION BY DATE(ANNUAL) 1998 to 2002 -- GROSS NATIONAL PRODUCT
GROSS NATIONAL PRODUCT
BILLIONS OF CURRENT DOLLARS
U.S. DEPARTMENT OF COMMERCE, BUREAU OF ECONOMIC ANALYSIS
'SURVEY OF CURRENT BUSINESS' AND OTHER MATERIALS
U.S. NATIONAL INCOME & PRODUCT ACCOUNTS - TABLE 1.9

SERIES PIDGNP : PRECISION BY DATE(AN

The Python dictionary `famedata` returned by `read_fame()` is easily converted to a Pandas `DataFrame`.

In [7]:
pd.DataFrame(data=famedata).T

,data,fame
$N,"[276.115288, 279.294713, 281.674017, 285.545, ...","{u'mdate': 830095207001, u'basis': 8, u'cdate'..."
BOPMERCH,"[-246.696, -346.022, -452.423, -427.215, -482....","{u'mdate': 830095207001, u'basis': 8, u'cdate'..."
GDP,"[8781.5, 9274.3, 9824.6, 10082.2, 10446.2]","{u'mdate': 830095207001, u'basis': 8, u'cdate'..."
GDP96C,"[8508.9, 8859.0, 9191.4, 9214.5, 9439.9]","{u'mdate': 830095207001, u'basis': 8, u'cdate'..."
GICV96,"[145.297, 154.106, 161.523, 168.123, 173.099]","{u'mdate': 830095207001, u'basis': 8, u'cdate'..."
GNP,"[8778.1, 9297.1, 9848.0, 10104.1, 10436.7]","{u'mdate': 830095207001, u'basis': 8, u'cdate'..."
GNP96C,"[8508.4, 8883.7, 9216.2, 9237.3, 9433.5]","{u'mdate': 830095207001, u'basis': 8, u'cdate'..."
HUSTS,"[1616.9, 1666.5, 1568.7, 1602.7, 1704.9]","{u'mdate': 830095207001, u'basis': 8, u'cdate'..."
HUSTS1,"[1271.4, 1302.4, 1230.9, 1273.3, 1358.6]","{u'mdate': 830095207001, u'basis': 8, u'cdate'..."
I,"[1538.7, 1636.7, 1755.4, 1586.0, 1593.2]","{u'mdate': 830095207001, u'basis': 8, u'cdate'..."


We will call `read_fame()` one last time to retrieve _all_ data from `driecon`.

In [8]:
famedata = qm.read_fame(dbname)
print("read_fame() returned {0} FAME objects from {1}.\n".format(len(famedata),dbname))

read_fame() returned 55 FAME objects from /opt/pkg/fame116linux/util/driecon.



We retrieve individual FAME data objects from the Python dictionary `famedata` using the `qomautils` function `get_fame_data()`.

In [9]:
print(qm.meta_to_string(famedata,'STRING_ATTRIBUTE_NAMES'))
qm.get(famedata,'STRING_ATTRIBUTE_NAMES')

SCALAR STRING_ATTRIBUTE_NAMES : NAMELIST



u'{DRISOURCE, STATUS, DRIUPDATED, DRIATTRIBUTES, DRILONGNAME}'

In [10]:
print(qm.meta_to_string(famedata,'DESC'))
qm.get(famedata,'DESC')

None -- Object Descriptions



1     US INTERNATIONAL TRANSACTIONS - BALANCE ON GOO...
2          NEW CONSTRUCTION PUT IN PLACE - TOTAL PUBLIC
3                 NEW CONSTRUCTION PUT IN PLACE - TOTAL
4              LAGGING INDICATORS COMPOSITE INDEX (BCI)
5              LEADING INDICATORS COMPOSITE INDEX (BCI)
6        RATIO, COINCIDENT INDEX TO LAGGING INDEX (BCI)
7                       DOW JONES: 65 COMPOSITE AVERAGE
8                      DOW JONES: 30 INDUSTRIAL AVERAGE
9       GOVT ISSUE: CONSTANT MATURITY, 30-YR (H15) - US
10    TREASURY BILL: SECONDARY, 3-MONTH BOND-EQUIVAL...
11                   CD: SECONDARY MKT, 1-MO YIELD - US
12    EXCHANGE RATE IN NEW YORK - FRENCH FRANC PER U...
13    EXCHANGE RATE IN NEW YORK - GERMAN MARK PER US...
14    EXCHANGE RATE IN NEW YORK - JAPANESE YEN PER U...
15    COMMERCIAL PAPER: NON-FINAN, 1-MO QUOTED YIELD...
16    COMMERCIAL PAPER: NON-FINAN, 1-DAY QUOTED YIEL...
17    EXCHANGE RATE IN NEW YORK - US CENTS PER BRITI...
18                FEDERAL DEBT OUTSTANDING - TOT

In [11]:
print(qm.meta_to_string(famedata,'GDP'))
qm.get(famedata,'GDP')

SERIES GDP : PRECISION BY DATE(ANNUAL) 1929 to 2002 -- GROSS DOMESTIC PRODUCT
GROSS DOMESTIC PRODUCT
BILLIONS OF CURRENT DOLLARS
U.S. DEPARTMENT OF COMMERCE, BUREAU OF ECONOMIC ANALYSIS
'SURVEY OF CURRENT BUSINESS' AND OTHER MATERIALS
U.S. NATIONAL INCOME & PRODUCT ACCOUNTS - TABLE 1.1,1.3,1.5,1.9



1929-12-31      103.7
1930-12-31       91.3
1931-12-31       76.6
1932-12-31       58.8
1933-12-31       56.4
1934-12-31       66.0
1935-12-31       73.3
1936-12-31       83.7
1937-12-31       91.9
1938-12-31       86.1
1939-12-31       92.0
1940-12-31      101.3
1941-12-31      126.7
1942-12-31      161.8
1943-12-31      198.4
1944-12-31      219.7
1945-12-31      223.0
1946-12-31      222.3
1947-12-31      244.4
1948-12-31      269.6
1949-12-31      267.7
1950-12-31      294.3
1951-12-31      339.5
1952-12-31      358.6
1953-12-31      379.9
1954-12-31      381.1
1955-12-31      415.2
1956-12-31      438.0
1957-12-31      461.5
1958-12-31      467.9
               ...   
1973-12-31     1385.5
1974-12-31     1501.0
1975-12-31     1635.2
1976-12-31     1823.9
1977-12-31     2031.4
1978-12-31     2295.9
1979-12-31     2566.4
1980-12-31     2795.6
1981-12-31     3131.3
1982-12-31     3259.2
1983-12-31     3534.9
1984-12-31     3932.7
1985-12-31     4213.0
1986-12-31     4452.9
1987-12-31

The Qoma utility function `close_hli()` closes the FAME environment.

In [12]:
if qm.close_hli() != 0:
    raise